# Frost Identification in Martian HiRISE Images

## Objective:

Embark on an independent research endeavor aimed at developing a robust classifier for distinguishing images of Martian terrain with frost. Leveraging Keras and Python, this project explores the dataset accessible at NASA's JPL Dataverse, specifically curated to study Mars' seasonal frost cycle and its impact on climate and surface evolution.

Import packages

In [1]:
import os
import os.path as op
import json
from pathlib import Path
import shutil
import logging
import numpy as np
from tqdm import tqdm
from skimage import io
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Data Preprocessing

The dataset is structured with images (in png format) and corresponding labels (in json files), organized into "subframes." Each subframe is a 5120x5120 pixel image, a crop of the original HiRISE images. These subframes have been further sliced into 299x299 "tiles," each annotated with labels ('frost' or 'background') for ML algorithms. The dataset consists of 214 subframes and a total of 119,920 tiles. Annotation information is stored in JSON files provided by human annotators.

In [2]:
# Logging configuration
logging.basicConfig(level=logging.INFO,
                    datefmt='%H:%M:%S',
                    format='%(asctime)s | %(levelname)-5s | %(module)-15s | %(message)s')

IMAGE_SIZE = (299, 299)  # All images contained in this dataset are 299x299 (originally, to match Inception v3 input size)
SEED = 17

# Head directory containing all image subframes. Update with the relative path of your data directory
data_head_dir = Path('./data')

# Find all subframe directories
subdirs = [Path(subdir.stem) for subdir in data_head_dir.iterdir() if subdir.is_dir()]
src_image_ids = ['_'.join(a_path.name.split('_')[:3]) for a_path in subdirs]

In [3]:
# Load train/val/test subframe IDs
def load_text_ids(file_path):
    """Simple helper to load all lines from a text file"""
    with open(file_path, 'r') as f:
        lines = [line.strip() for line in f.readlines()]
    return lines

# Load the subframe names for the three data subsets
train_ids = load_text_ids('./train_source_images.txt')
validate_ids = load_text_ids('./val_source_images.txt')
test_ids = load_text_ids('./test_source_images.txt')

# Generate a list containing the dataset split for the matching subdirectory names
subdir_splits = []
for src_id in src_image_ids:
    if src_id in train_ids:
        subdir_splits.append('train')
    elif src_id in validate_ids:
        subdir_splits.append('validate')
    elif(src_id in test_ids):
        subdir_splits.append('test')
    else:
        logging.warning(f'{src_id}: Did not find designated split in train/validate/test list.')
        subdir_splits.append(None)

## (b) Data Exploration and Pre-processing:

Image tiles are categorized into 'background' and 'frost' classes for binary classification, with individual tiles serving as data points for the final project.
Files for data splitting into train, test, and validation are available. However, an improved version of these files will be provided upon repository creation.

In [4]:
import random
import tensorflow as tf
from PIL import Image 

def load_and_preprocess(img_loc, label):
    def _inner_function(img_loc, label):
        # Convert tensor to native type
        img_loc_str = img_loc.numpy().decode('utf-8')
        
        # Load image using PIL and convert to RGB
        img = Image.open(img_loc_str).convert('RGB')
        
        # Convert PIL image to numpy array
        img = np.array(img)
        img = tf.image.resize(img, [299, 299])
        
        # Normalize the image to the [0, 1] range
        img = img / 255.0

        # Convert label to integer (assuming binary classification)
        label = 1 if label.numpy().decode('utf-8') == 'frost' else 0
        
        return img, label

    # Wrap the Python function
    X, y = tf.py_function(_inner_function, [img_loc, label], [tf.float32, tf.int64])
    
    # Set the shape of the tensors
    X.set_shape([299, 299, 3])
    y.set_shape([])  # Scalar label
    
    return X, y

def load_subdir_data(dir_path, image_size, seed=None):
    
    """Helper to create a TF dataset from each image subdirectory"""
    
    # Grab only the classes that (1) we want to keep and (2) exist in this directory
    tile_dir = dir_path / Path('tiles')
    label_dir = dir_path /Path('labels')
    
    loc_list = []
    
    for folder in os.listdir(tile_dir):
        if os.path.isdir(os.path.join(tile_dir, folder)):
            for file in os.listdir(os.path.join(tile_dir, folder)):
                if file.endswith(".png"):
                    loc_list.append((os.path.join(os.path.join(tile_dir, folder), file), folder))

    return loc_list

# Loop over all subframes, loading each into a list
tf_data_train, tf_data_test, tf_data_val = [], [], []
tf_dataset_train, tf_dataset_test, tf_dataset_val = [], [], []

# Update the batch and buffer size as per your model requirements
buffer_size = 64
batch_size = 32

for subdir, split in zip(subdirs, subdir_splits):
    full_path = data_head_dir / subdir
    if split=='validate':
        tf_data_val.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='train':
        tf_data_train.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
    elif split=='test':
        tf_data_test.extend(load_subdir_data(full_path, IMAGE_SIZE, SEED))
        
random.shuffle(tf_data_train)
img_list, label_list = zip(*tf_data_train)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_train = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_train = tf_dataset_train.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_train = tf_dataset_train.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_val)
img_list, label_list = zip(*tf_data_val)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_val = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_val = tf_dataset_val.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_val = tf_dataset_val.shuffle(buffer_size=buffer_size).batch(batch_size) 

random.shuffle(tf_data_test)
img_list, label_list = zip(*tf_data_test)
img_list_t = tf.convert_to_tensor(img_list)
lb_list_t = tf.convert_to_tensor(label_list)

tf_dataset_test = tf.data.Dataset.from_tensor_slices((img_list_t, lb_list_t))
tf_dataset_test = tf_dataset_test.map(load_and_preprocess, num_parallel_calls=tf.data.experimental.AUTOTUNE)
tf_dataset_test = tf_dataset_test.shuffle(buffer_size=buffer_size).batch(batch_size) 

21:47:18 | INFO  | utils           | NumExpr defaulting to 8 threads.


## (c). Training CNN + MLP

#### (i).

Implement empirical regularization techniques, including cropping, random zooming, rotation, flipping, contrast adjustments, and translation for image augmentation, utilizing tools such as OpenCV.

In [5]:
import numpy as np
import cv2
import tensorflow as tf

def augment_batch(batch_images):
    processed_images = []

    for img in batch_images.numpy():
        if np.random.rand() > 0.5:
            img = cv2.flip(img, 1)

        angle = np.random.randint(-30, 30)  
        h, w = img.shape[:2]
        M = cv2.getRotationMatrix2D((w / 2, h / 2), angle, 1)
        img = cv2.warpAffine(img, M, (w, h))

        min_zoom_factor = 0.8
        max_zoom_factor = 1.2
        zoom_factor = np.random.uniform(min_zoom_factor, max_zoom_factor)
        h, w = img.shape[:2]
        new_h = int(h * zoom_factor)
        new_w = int(w * zoom_factor)
        img = cv2.resize(img, (new_w, new_h))

        start_x = max(0, (new_w - w) // 2)
        start_y = max(0, (new_h - h) // 2)
        img = img[start_y:start_y + h, start_x:start_x + w]

        contrast_factor = 1.0 + np.random.uniform(-0.5, 0.5) 
        img = np.clip(contrast_factor * img, 0, 255).astype(np.uint8)

        tx, ty = np.random.randint(-30, 30), np.random.randint(-30, 30) 
        translation_matrix = np.float32([[1, 0, tx], [0, 1, ty]])
        img = cv2.warpAffine(img, translation_matrix, (w, h))

        processed_images.append(img)

    return np.stack(processed_images, axis=0)

def apply_augmentation_to_batch(images, labels):
    processed_images = tf.py_function(augment_batch, [images], tf.uint8)
    processed_images.set_shape([None, 299, 299, 3])  
    return processed_images, labels
tf_dataset_train_augmented = tf_dataset_train.map(apply_augmentation_to_batch, num_parallel_calls=tf.data.experimental.AUTOTUNE)

#### (ii).

Train a three-layer CNN followed by a dense layer, allowing flexibility in choosing kernel sizes, layer depths, and the number of neurons in the dense layer (MLP). Incorporate ReLU activation functions, softmax function, batch normalization, a dropout rate of 30%, L2 regularization, and the ADAM optimizer. Employ cross-entropy loss and conduct training for a minimum of 20 epochs with early stopping using the validation set. Save the network parameters with the lowest validation error.

In [ ]:
from tensorflow.keras import models
import numpy as np
from tensorflow.keras.regularizers import l2 
from tensorflow.keras import models  # Add this import statement
from tensorflow.keras import layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt

def build_cnn_model(input_shape, num_classes):
    model = models.Sequential([
        # First Convolutional Layer
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=input_shape, kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Second Convolutional Layer
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Third Convolutional Layer
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten Layer
        layers.Flatten(),
        
        # Dense Layer (MLP)
        layers.Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        
        # Output Layer
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

input_shape = (299, 299, 3)  
num_classes = 2 
learning_rate = 0.0001
batch_size = 32
epochs = 20

model = build_cnn_model(input_shape, num_classes)

model.compile(optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(tf_dataset_train_augmented, epochs=epochs, validation_data=tf_dataset_val, callbacks=[early_stopping])

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

Epoch 1/20
928/928 [==============================] - 1686s 2s/step - loss: 1.2850 - accuracy: 0.5611 - val_loss: 3.0357 - val_accuracy: 0.3218
Epoch 2/20
928/928 [==============================] - 1560s 2s/step - loss: 1.1130 - accuracy: 0.5797 - val_loss: 2.7178 - val_accuracy: 0.3184
Epoch 3/20
928/928 [==============================] - 1608s 2s/step - loss: 1.0454 - accuracy: 0.5863 - val_loss: 5.3819 - val_accuracy: 0.3218
Epoch 4/20
928/928 [==============================] - 2271s 2s/step - loss: 1.0094 - accuracy: 0.5887 - val_loss: 6.1318 - val_accuracy: 0.3218
Epoch 5/20
928/928 [==============================] - 4905s 5s/step - loss: 0.9760 - accuracy: 0.5887 - val_loss: 3.3406 - val_accuracy: 0.3218
Epoch 6/20
928/928 [==============================] - 5960s 6s/step - loss: 0.9496 - accuracy: 0.5943 - val_loss: 3.1880 - val_accuracy: 0.3218
Epoch 7/20
928/928 [==============================] - 5716s 6s/step - loss: 0.9153 - accuracy: 0.6035 - val_loss: 5.3343 - val_accuracy:

#### (iii).

Report Precision, Recall, and F1 score for the developed model.

In [8]:
from sklearn.metrics import classification_report
test_predictions = model.predict(tf_dataset_test)
test_predictions_labels = np.argmax(test_predictions, axis=1) 
test_labels = np.concatenate([y.numpy() for _, y in tf_dataset_test])
report = classification_report(test_labels, test_predictions_labels)
print(report)

442/442 [==============================] - 124s 281ms/step
              precision    recall  f1-score   support

           0       0.35      0.78      0.48      4932
           1       0.66      0.23      0.34      9187

    accuracy                           0.42     14119
   macro avg       0.50      0.50      0.41     14119
weighted avg       0.55      0.42      0.39     14119

